In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [9]:
from sklearn.linear_model import Lasso

lasso= Lasso(alpha= 0.01)
lasso.fit(X_sm_tl, y_sm_tl)

Lasso(alpha=0.01)

In [10]:
selected_features= np.where(lasso.coef_ != 0)[0]
X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

print(f"Number of features selected: {len(selected_features)}")

Number of features selected: 26


In [11]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    y_pred_prob= model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))
    print('Precision:', precision_score(y_test, y_pred, zero_division=0))
    print('Accuracy:', accuracy_score(y_test, y_pred))
    if y_pred_prob is not None:
        print('AUC:', roc_auc_score(y_test, y_pred_prob))
    print('-' * 50)

--- LogisticRegression ---

In [12]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

train_and_evaluate_model(lr, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[6374 2143]
 [ 183  729]]
              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.25      0.80      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.77      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.2538300835654596
Accuracy: 0.7533142432919716
AUC: 0.8511920946548595
--------------------------------------------------


--- DecisionTreeClassifier ---

In [13]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

train_and_evaluate_model(dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[7744  773]
 [ 671  241]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.91      8517
           1       0.24      0.26      0.25       912

    accuracy                           0.85      9429
   macro avg       0.58      0.59      0.58      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.23767258382642997
Accuracy: 0.8468554459645774
AUC: 0.5860358102165123
--------------------------------------------------


--- Tuned - DecisionTreeClassifier ---

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9408415474451101


In [15]:
tuned_dt= gs_dt.best_estimator_
train_and_evaluate_model(tuned_dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8073  444]
 [ 690  222]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      8517
           1       0.33      0.24      0.28       912

    accuracy                           0.88      9429
   macro avg       0.63      0.60      0.61      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.3333333333333333
Accuracy: 0.8797327394209354
AUC: 0.7498809141263397
--------------------------------------------------


--- KNeighborsClassifier ---

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
train_and_evaluate_model(knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[7058 1459]
 [ 398  514]]
              precision    recall  f1-score   support

           0       0.95      0.83      0.88      8517
           1       0.26      0.56      0.36       912

    accuracy                           0.80      9429
   macro avg       0.60      0.70      0.62      9429
weighted avg       0.88      0.80      0.83      9429

Precision: 0.26051697921946276
Accuracy: 0.803054406617881
AUC: 0.7651004106338407
--------------------------------------------------


--- Tuned - KNeighborsClassifier ---

In [17]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9311578759522989


In [18]:
tuned_knn= gs_knn.best_estimator_
train_and_evaluate_model(tuned_knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8035  482]
 [ 702  210]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.30      0.23      0.26       912

    accuracy                           0.87      9429
   macro avg       0.61      0.59      0.60      9429
weighted avg       0.86      0.87      0.87      9429

Precision: 0.30346820809248554
Accuracy: 0.8744299501537809
AUC: 0.6458877266107621
--------------------------------------------------


--- GaussianNB ---

In [19]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
train_and_evaluate_model(nb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[4030 4487]
 [  81  831]]
              precision    recall  f1-score   support

           0       0.98      0.47      0.64      8517
           1       0.16      0.91      0.27       912

    accuracy                           0.52      9429
   macro avg       0.57      0.69      0.45      9429
weighted avg       0.90      0.52      0.60      9429

Precision: 0.15626175253854832
Accuracy: 0.5155371725527628
AUC: 0.8114495982235735
--------------------------------------------------


--- SVM ---

In [20]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
train_and_evaluate_model(svc, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[7797  720]
 [ 703  209]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      8517
           1       0.22      0.23      0.23       912

    accuracy                           0.85      9429
   macro avg       0.57      0.57      0.57      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.22497308934337998
Accuracy: 0.8490826174567823
AUC: 0.7332988177411945
--------------------------------------------------


--- Random Forest ---

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
train_and_evaluate_model(rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8157  360]
 [ 747  165]]
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8517
           1       0.31      0.18      0.23       912

    accuracy                           0.88      9429
   macro avg       0.62      0.57      0.58      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.3142857142857143
Accuracy: 0.8825962456251989
AUC: 0.8026335100696439
--------------------------------------------------


In [22]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best Precision Score: 0.9747355169512488


In [23]:
tuned_rf= gs_tuned_rf.best_estimator_
train_and_evaluate_model(tuned_rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8260  257]
 [ 744  168]]
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.40      0.18      0.25       912

    accuracy                           0.89      9429
   macro avg       0.66      0.58      0.60      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.3952941176470588
Accuracy: 0.8938381588715665
AUC: 0.8195885061662023
--------------------------------------------------


--- AdaBoost ---

In [24]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
train_and_evaluate_model(ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[7830  687]
 [ 509  403]]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.37      0.44      0.40       912

    accuracy                           0.87      9429
   macro avg       0.65      0.68      0.67      9429
weighted avg       0.88      0.87      0.88      9429

Precision: 0.3697247706422018
Accuracy: 0.8731572807296638
AUC: 0.8399706649652192
--------------------------------------------------


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9611136817860977


In [26]:
tuned_ada= gs_ada.best_estimator_
train_and_evaluate_model(tuned_ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8174  343]
 [ 644  268]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      8517
           1       0.44      0.29      0.35       912

    accuracy                           0.90      9429
   macro avg       0.68      0.63      0.65      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.4386252045826514
Accuracy: 0.8953229398663697
AUC: 0.8474920000041197
--------------------------------------------------


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
train_and_evaluate_model(grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8020  497]
 [ 590  322]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      8517
           1       0.39      0.35      0.37       912

    accuracy                           0.88      9429
   macro avg       0.66      0.65      0.65      9429
weighted avg       0.88      0.88      0.88      9429

Precision: 0.39316239316239315
Accuracy: 0.8847173613320607
AUC: 0.845163195281264
--------------------------------------------------


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.971352205142504


In [29]:
tuned_grb= gs_grb.best_estimator_
train_and_evaluate_model(tuned_grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8273  244]
 [ 690  222]]
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.48      0.24      0.32       912

    accuracy                           0.90      9429
   macro avg       0.70      0.61      0.63      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.47639484978540775
Accuracy: 0.9009438964895535
AUC: 0.8489249571033373
--------------------------------------------------


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
train_and_evaluate_model(xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8368  149]
 [ 772  140]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.70      0.57      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4844290657439446
Accuracy: 0.9023226216990137
AUC: 0.8378580815664852
--------------------------------------------------


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
train_and_evaluate_model(tuned_xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9687437596374279
[[8237  280]
 [ 688  224]]
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.44      0.25      0.32       912

    accuracy                           0.90      9429
   macro avg       0.68      0.61      0.63      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.4444444444444444
Accuracy: 0.8973379997878884
AUC: 0.8437094464322129
--------------------------------------------------


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
train_and_evaluate_model(lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.509984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5197
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[[8376  141]
 [ 770  142]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.50      0.16      0.24       912

    accuracy                           0.90      9429
   macro avg       0.71      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5017667844522968
Accuracy: 0.9033831795524446
AUC: 0.8451673150087854
--------------------------------------------------


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
train_and_evaluate_model(tuned_lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 300, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.9800382832044241
[LightGBM] [Warning] min_data_in_leaf is set=

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(learning_rate= 0.1, depth= 6, iterations= 100, random_state= 15, verbose= 0)
train_and_evaluate_model(cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[8326  191]
 [ 716  196]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.51      0.21      0.30       912

    accuracy                           0.90      9429
   macro avg       0.71      0.60      0.63      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.5064599483204134
Accuracy: 0.903807402693817
AUC: 0.851240630194719
--------------------------------------------------


In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
train_and_evaluate_model(tuned_cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'depth': 3, 'iterations': 50, 'learning_rate': 1}
Best Precision Score: 0.9792057332157496
[[8387  130]
 [ 770  142]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.52      0.16      0.24       912

    accuracy                           0.90      9429
   macro avg       0.72      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5220588235294118
Accuracy: 0.9045497931912185
AUC: 0.8459492907889072
--------------------------------------------------


In [36]:
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300


--- Visualisation ---

In [37]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         0.522059
16                  Tuned LGBM Precision:         0.508418
17                    CatBoost Precision:         0.506460
15                        LGBM Precision:         0.501767
13                         XGB Precision:         0.484429
12      Tuned GradientBoosting Precision:         0.476395
14                   Tuned XGB Precision:         0.444444
10              Tuned AdaBoost Precision:         0.438625
8          Tuned Random Forest Precision:         0.395294
11            GradientBoosting Precision:         0.393162
9                     AdaBoost Precision:         0.369725
2          Tuned Decision Tree Precision:         0.333333
7                Random Forest Precision:         0.314286
4   Tuned KNeighborsClassifier Precision:         0.303468
3         KNeighborsClassifier Precision:         0.260517
0          Logistic Regression Precision:         0.2538

In [38]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
18              Tuned CatBoost Accuracy:        0.904550
17                    CatBoost Accuracy:        0.903807
16                  Tuned LGBM Accuracy:        0.903807
15                        LGBM Accuracy:        0.903383
13                         XGB Accuracy:        0.902323
12      Tuned GradientBoosting Accuracy:        0.900944
14                   Tuned XGB Accuracy:        0.897338
10              Tuned AdaBoost Accuracy:        0.895323
8          Tuned Random Forest Accuracy:        0.893838
11            GradientBoosting Accuracy:        0.884717
7                Random Forest Accuracy:        0.882596
2          Tuned Decision Tree Accuracy:        0.879733
4   Tuned KNeighborsClassifier Accuracy:        0.874430
9                     AdaBoost Accuracy:        0.873157
6                          SVM Accuracy:        0.849083
1                Decision Tree Accuracy:        0.846855
3         KNeighborsClassifier 

In [39]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
17                    CatBoost AUC:   0.851241
0          Logistic Regression AUC:   0.851192
12      Tuned GradientBoosting AUC:   0.848925
10              Tuned AdaBoost AUC:   0.847492
18              Tuned CatBoost AUC:   0.845949
15                        LGBM AUC:   0.845167
11            GradientBoosting AUC:   0.845163
16                  Tuned LGBM AUC:   0.844766
14                   Tuned XGB AUC:   0.843709
9                     AdaBoost AUC:   0.839971
13                         XGB AUC:   0.837858
8          Tuned Random Forest AUC:   0.819589
5                   GaussianNB AUC:   0.811450
7                Random Forest AUC:   0.802634
3         KNeighborsClassifier AUC:   0.765100
2          Tuned Decision Tree AUC:   0.749881
6                          SVM AUC:   0.733299
4   Tuned KNeighborsClassifier AUC:   0.645888
1                Decision Tree AUC:   0.586036
